In [3]:
import pandas as pd
import numpy as np
from datetime import timedelta

# Load data
csv_file_path = "/Users/balmeru/Downloads/QQQQ*.csv"
df = pd.read_csv(csv_file_path)
df['rdq'] = pd.to_datetime(df['rdq'], errors='coerce')

df['txdbq'] = df['txdbq'].fillna(0)
df['dvpq'] = df['dvpq'].fillna(0)

df.loc[:, "income_good"] = False
df.loc[~df["ibq"].isna(), "income_good"] = True

#1
df.loc[:, "se1_good"] = False
df.loc[~df["teqq"].isna(), "se1_good"] = True

#2
df.loc[:, "se2_good"] = False
df.loc[~df["ceqq"].isna() & ~df["pstkq"].isna(), "se2_good"] = True

#3
df.loc[:, "se3_good"] = False
df.loc[~df["atq"].isna() & ~df["ltq"].isna(), "se3_good"] = True

# Fiscal year changes
fyr_changes = df.groupby('tic')['fyr'].nunique()
tickers_with_changes = fyr_changes[fyr_changes > 1].index
df.loc[:, 'fyr_change'] = False
df.loc[df['tic'].isin(tickers_with_changes), 'fyr_change'] = True

# Filter out cases where fyr_change is False and conditions for income_good and se goods are met
df_filtered = df[(df['fyr_change'] == False) & (df['income_good'] == True) & ((df['se1_good'] == True) | (df['se2_good'] == True) | (df['se3_good'] == True))]

target_year = 1984
reference_year = 1983
expected_dates = []
for tic in df_filtered['tic'].unique():
    tic_data = df_filtered[
        (df_filtered['tic'] == tic) & 
        (df_filtered['fyearq'] == reference_year)
    ]
    
    for quarter in range(1, 5):
        quarter_data = tic_data[
            (tic_data['fqtr'] == quarter)
        ]
        
        if not quarter_data.empty:
            reference_date = quarter_data['rdq'].iloc[0]
            expected_date = reference_date + timedelta(weeks=52)
        else:
            expected_date = np.nan  
        
        expected_dates.append({
            'quarter': f"1984Q{quarter}",  
            'tic': tic,                   
            'expected_announcement_date': expected_date  
        })

expected_dates_df = pd.DataFrame(expected_dates)
expected_dates_df['quarter'] = pd.Categorical(expected_dates_df['quarter'], categories=['1984Q1', '1984Q2', '1984Q3', '1984Q4'], ordered=True)

pivot_table = expected_dates_df.pivot_table(
    index='quarter',   
    columns='tic',    
    values='expected_announcement_date'  
)

pivot_table_cleaned = pivot_table.dropna(axis=1, how='all')
print("Pivot Table corrected for both early and late reporting:")
print(pivot_table_cleaned)



/var/folders/ds/rfyxvvb945vdt8znlkx31w1m00cj2y/T/ipykernel_4784/3732293425.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['txdbq'].fillna(0, inplace=True)
/var/folders/ds/rfyxvvb945vdt8znlkx31w1m00cj2y/T/ipykernel_4784/3732293425.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as

Pivot Table corrected for both early and late reporting:
tic          0033A      0131A      0141A      0173A      0223B      0571B  \
quarter                                                                     
1984Q1  1984-04-23 1984-04-25 1984-04-20 1984-08-10 1984-04-23 1984-04-30   
1984Q2  1984-07-25 1984-08-14 1984-07-24 1984-11-14 1984-08-01 1984-07-25   
1984Q3  1984-10-29 1984-11-05 1984-10-23 1985-02-15 1984-10-24 1984-10-26   
1984Q4  1985-02-19 1985-02-15 1985-01-29 1985-06-06 1985-01-30 1985-03-13   

tic          0575B      0602B      0780B      0949B  ...       ZENL  \
quarter                                              ...              
1984Q1  1983-11-14        NaT 1984-04-25 1984-03-15  ...        NaT   
1984Q2  1984-02-03 1984-09-14 1984-07-24 1984-05-29  ... 1984-08-14   
1984Q3  1984-05-04 1984-11-30 1984-10-22 1984-08-23  ...        NaT   
1984Q4  1984-08-29 1985-05-21 1985-01-24 1984-12-18  ... 1985-03-15   

tic           ZENT       ZGCO       ZIAD      ZIM.1  

/var/folders/ds/rfyxvvb945vdt8znlkx31w1m00cj2y/T/ipykernel_4784/3732293425.py:65: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot_table = expected_dates_df.pivot_table(


In [ ]:
pivot_table_cleaned.to_csv('expected_dates_1984.csv', index=True)
